<h1> Model Training - Multilayer Perceptron</h1>

In [35]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import metrics
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split
from joblib import dump, load

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)

In [36]:
df = pd.read_csv('imoveis.csv')

In [37]:
df.head()

,local,preco,metragem,quartos,garagens,banheiros,bairro
0,"Rua Maria Thereza Gonçalves, Umbara",399900,106,3.0,2.0,2,76
1,"Travessa Paulo Ribeiro, Sitio Cercado",765190,319,5.0,3.0,4,70
2,"Uberaba, Curitiba",2350000,500,5.0,7.0,5,75
3,"Rua Ângelo Greca, Atuba",737000,144,4.0,NaN,4,7
4,"Rua João Fonseca Mercer, Atuba",1140000,276,5.0,3.0,5,7


In [38]:
x = df.drop("preco", axis=1).drop("local", axis=1)
y = df["preco"]

In [39]:
scaler = MinMaxScaler()
scaler_preco = MinMaxScaler()

features = [['metragem', 'quartos', 'garagens', 'banheiros', 'bairro']]
for feature in features:
    x[feature] = scaler.fit_transform(x[feature])

y = scaler_preco.fit_transform(y.values.reshape(-1,1))

In [40]:
x.fillna(0, inplace=True)
Y =y
X = x.to_numpy()

In [41]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [42]:
regressor = MLPRegressor(random_state=10, max_iter=1500, early_stopping=True, validation_fraction=0.2)
regressor.fit(x_treinamento, y_treinamento.ravel())


MLPRegressor(early_stopping=True, max_iter=1500, random_state=10,
             validation_fraction=0.2)

In [43]:
previsoes = regressor.predict(x_teste)

In [44]:
print("MAE:",scaler_preco.inverse_transform(metrics.mean_absolute_error(y_teste, previsoes).reshape(-1,1))[0][0])
print("MSE:", scaler_preco.inverse_transform(metrics.mean_squared_error(y_teste, previsoes).reshape(-1,1))[0][0])
print("MSLE:",scaler_preco.inverse_transform(metrics.mean_squared_log_error(y_teste, previsoes).reshape(-1,1))[0][0])
print("R2:", metrics.r2_score(y_teste, previsoes))

MAE: 348947.15955220227
MSE: 88568.05948821273
MSLE: 85393.58021281747
R2: 0.4630140052087355


In [45]:
scaler_preco.inverse_transform(previsoes.reshape(-1, 1))[:3].T

array([[750657.81292769, 915721.26116992, 888212.93047651]])

In [46]:
scaler_preco.inverse_transform(y_teste.reshape(-1, 1))[:3].T

array([[ 780000., 1400000.,  730000.]])

In [47]:
area = 247.6
quartos = 4
garagens = 4
banheiros = 3
bairro = 61 #Santa Felicidade
valores = scaler.transform([[area, quartos, garagens, banheiros, bairro]])
casa = regressor.predict(valores)

print("Preço estimado: R$", scaler_preco.inverse_transform(casa.reshape(-1,1))[0][0])

Preço estimado: R$ 944717.3334670621


In [48]:
with open("models/MLP.mod", 'wb') as fo:  
    dump(regressor, fo)